### Target:
    
    - Add LR Scheduler ReduceLROnPlateau.

### Results:

    - Model has 7979 parameters.

    - Train Accuracy = 98.85

    - Test Accuracy = 99.44

### Analysis:

    - Target achieved: crossed 99.4% validation accuracy 3 times (epochs 11, 14 and 15)


In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print("Device Selected:", device)

Device Selected: mps


In [3]:
SEED = 1
torch.manual_seed(SEED)

if device == 'cuda':
    torch.cuda.manual_seed(SEED)

In [4]:
# Train data transformations
train_transforms = transforms.Compose([
    transforms.RandomRotation((-7.0, 7.0), fill=(0,)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# Test data transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [5]:
train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transforms)

In [6]:
batch_size = 64

kwargs = {'batch_size': batch_size, 'num_workers': 4, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, **kwargs)

In [7]:
from model import Model4 as Net

model = Net()
model.summary(input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 9, 28, 28]              81
           Dropout-2            [-1, 9, 28, 28]               0
              ReLU-3            [-1, 9, 28, 28]               0
       BatchNorm2d-4            [-1, 9, 28, 28]              18
            Conv2d-5           [-1, 10, 28, 28]             810
           Dropout-6           [-1, 10, 28, 28]               0
              ReLU-7           [-1, 10, 28, 28]               0
         MaxPool2d-8           [-1, 10, 14, 14]               0
       BatchNorm2d-9           [-1, 10, 14, 14]              20
           Conv2d-10           [-1, 16, 14, 14]           1,440
          Dropout-11           [-1, 16, 14, 14]               0
             ReLU-12           [-1, 16, 14, 14]               0
        MaxPool2d-13             [-1, 16, 7, 7]               0
      BatchNorm2d-14             [-1, 1

In [8]:
from utils import train, test

In [9]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=0, verbose=True, factor=0.5)

criterion = F.nll_loss
num_epochs = 15

for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train(model, device, train_loader, optimizer, criterion)
  test_loss = test(model, device, test_loader, criterion)
  scheduler.step(test_loss)

Epoch 1


  0%|          | 0/938 [00:00<?, ?it/s]

Train: Batch Loss=2.2902 Batch_id=937 Average Loss=2.3021 Accuracy=12.76: 100%|██████████| 938/938 [00:15<00:00, 59.72it/s]


Test set: Average loss: 2.2622, Accuracy: 1030/10000 (10.30%)

Epoch 2


Train: Batch Loss=2.3023 Batch_id=937 Average Loss=2.2839 Accuracy=13.64: 100%|██████████| 938/938 [00:13<00:00, 69.88it/s]


...

Test set: Average loss: 2.2739, Accuracy: 1035/10000 (10.35%)

Epoch 00007: reducing learning rate of group 0 to 3.1250e-04.
Epoch 8


Train: Batch Loss=2.2895 Batch_id=937 Average Loss=2.2689 Accuracy=14.55: 100%|██████████| 938/938 [00:13<00:00, 69.06it/s]


Test set: Average loss: 2.2749, Accuracy: 740/10000 (7.40%)

Epoch 00008: reducing learning rate of group 0 to 1.5625e-04.
Epoch 9


Train: Batch Loss=2.3384 Batch_id=405 Average Loss=2.2704 Accuracy=14.49:  43%|████▎     | 406/938 [00:06<00:09, 58.67it/s]


KeyboardInterrupt: 

In [ ]:
from utils import plot_stats
plot_stats()

In [ ]:
from utils import show_incorrect
show_incorrect() # Predicted vs Actual